In [64]:
import pandas as pd
import yaml

## Plan

### Here are the steps you need to complete to make this function:
-  [x] open the .yaml file with tree properties
    - [x] use these tree properties to assign a death year for each tree based on (1) species and (2) age/year planted
    
- [x] develop the architecture of the .yaml file with strategies. What will this look like? What information will it hold? How flexible/rigid will it be (at first)?

- [x] first off, based on the initial data, determine the death year of all of the trees. append this to a column 'death year' in the new dataframe

- [x]  start by making a function that simply creates the new event dataframe assuming no strategies are implemented. I.e. all the rows will have status of either "plant" or "replant." End year is determined by the death year, and replant is simply determined by the death year + 1.

- [ ]  add to that function the ability to add pruning, so the status will now be "plant," "replant," or "prune"
    * pruning will have to be implemented with some sort of year crieteria (in what year, or even better, at what age, should farmers prune?)
    * with the new row that represents the pruning event, death year carries over but is altered slightly (because we are working under the assumption that pruning increases production life)
    * unless another event is called, end year is death year
    
-  [ ] add to that function the ability to add intercropping.
    * intercropping will be implemented with (1) a year criteria and (2) a proportion criteria (what proportion of the trees will be replanted). 
    * if the trees in the originaal dataset reach the year or age criteria, the original row will become inactive because the 'end year' has been reached. The row will then be split into two rows (but both still show the same ID number, which is how we identify them as the same plot). The first row will be the trees that were left alone, and so it will be the exact same except that is has proportionally less cuerdas (the death year will carry over from the original). The second row will have the trees which were replanted. Their


In [127]:
def openYaml(yamlFilePath : str) -> dict: 
    yamlFile = open(yamlFilePath)
    parsed = yaml.load(yamlFile, Loader =yaml.FullLoader)
    return(parsed)

In [128]:
# open the yaml files  to assign attirbutes to the sim
treeAttributes = openYaml("data/trees.yml")
strategyAttributes = openYaml("intervention/strategy.yml")

In [129]:
# initialData = pd.read_csv("data/demoData.csv")
upData = pd.read_csv("data/demoDataUpdated.csv")
upData.head()

,plotID,farmerName,treeType,numCuerdas,ageOfTrees,yearPlanted
0,0,Estefanía Cazares,catuai,9,9,2011
1,1,Aldo Linares,catuai,1,2,2018
2,2,Aldo Linares,borbon,1,20,2000
3,3,Linda Gabriel Portillo Herrera,catuai,4,7,2013
4,4,Linda Gabriel Portillo Herrera,borbon,10,27,1993


In [130]:
# yearPlanted = 2020 - initialData['ageOfTrees']
# initialData['yearPlanted'] = yearPlanted
# print(initialData.keys())
# new = initialData.rename(columns={'Unnamed: 0': 'plotID'})
# new.to_csv("data/demoDataUpdated.csv",  index = False)

In [131]:
def transformData(farmData, treeAttributes, strategyAttributes):
    print("something")

In [159]:
# transformedTemplate = pd.DataFrame(columns=["plotID", "farmerName", "treeType", 
#                                             "numCuerdas", "status", "startYear",
#                                            "deathYear"])
currentYear = 2020
simulationYears = 30

endYear = currentYear + simulationYears
# first, just replant

for i in range(len(upData)):
    _plotID = upData["plotID"][i]
    _farmerName = upData["farmerName"][i]
    _treeType = upData["treeType"][i]
    _numCuerdas =  upData["numCuerdas"][i]
    _startYear = upData["yearPlanted"][i]
    # assume all are planted for initialization
    _status = "plant" 
    
    _treeAge = currentYear - _startYear
    
    _altOrth = [treeAttributes[item]["altOrth"] for item in treeAttributes]
    if (_treeType in treeAttributes.keys() or _treeType in _altOrth):
        
        #  isolate dict
        _treeAttributes = treeAttributes[_treeType]
        # isolate smaller variables and dicts
        _cuerdaHarvestCap  = _treeAttributes["cuerdaHarvestCap"]
        _firstHarvest = _treeAttributes["firstHarvest"]
        _fullHarvest = _treeAttributes["fullHarvest"]
        _descentHarvest = _treeAttributes["descentHarvest"]
        _death = _treeAttributes["death"]
        
        # calculate death year
        _yearsTillDeath = _death["year"] - _treeAge
        _deathYear = currentYear + _yearsTillDeath
        
        initialRow = pd.DataFrame([[_plotID, _farmerName, _treeType, _numCuerdas, _status, 
                                  _startYear, _deathYear]], columns=["plotID", "farmerName", "treeType", 
                                            "numCuerdas", "status", "startYear",
                                           "deathYear"])
        if (i == 0):
            transformedTemplate = initialRow
            
        else:
            transformedTemplate = pd.concat([transformedTemplate, initialRow], ignore_index=True)
            #transformedTemplate  = transformedTemplate.reset_index(drop = True, inplace = True)
        # now you've transformed the original data
        # now you should be iterating through transformed data
        
        
       
        
#         if strategyAttributes["replant"]["isReplant"] ==  True:
        _replantYear = (_deathYear + 1)
        
        if strategyAttributes["prune"]["isPrune"] ==  True:
            _pruneAge = strategyAttributes["prune"]["age"]
        else:
            _pruneAge = None
            
        if strategyAttributes["intercrop"] == True:
            _intercropAge = strategyAttributes["prune"]["age"]
        else:
            _intercropAge = None
        
        
         # create a row to check against to see if the program needs to continue
        checkRow = initialRow
        
        _simYear = currentYear
        _simTreeAge = _treeAge
        
        
        while (_simYear < endYear):
           #  isolate dict
            _deathYear = checkRow["deathYear"][0]
            
            
                
            if (_simYear == _deathYear):
                # update death year
                _simTreeAge = -1
                _deathYear = (_simYear +  1) + _death["year"]
                _status = "replant"
                nextRow = pd.DataFrame([[_plotID, _farmerName, _treeType, _numCuerdas, _status, _replantYear, _deathYear]], 
                                    columns=["plotID", "farmerName","treeType", "numCuerdas", "status", "startYear","deathYear"])
                transformedTemplate  = pd.concat([transformedTemplate, nextRow],  ignore_index=True)
                checkRow = nextRow
                _simYear += 1
                _simTreeAge += 1
                #print(_simYear)
                continue
                    
          
                
            elif (_pruneAge):
                if (_simTreeAge == _pruneAge):
                    adjustedDeathYear = (checkRow["deathYear"][0]) + 4
                    _pruneYear = _simYear
                    _status = "prune"
                    nextRow = pd.DataFrame([[_plotID, _farmerName, _treeType, _numCuerdas, _status, _pruneYear, adjustedDeathYear]], 
                                           columns=["plotID", "farmerName","treeType", "numCuerdas", "status", "startYear","deathYear"])
                    transformedTemplate  = pd.concat([transformedTemplate, nextRow],  ignore_index=True)
                    checkRow  = nextRow
                    _simYear += 1
                    _simTreeAge += 1
                    continue
                    
                else:
                    _simYear += 1
                    _simTreeAge += 1
                    continue

            else:
                _simYear += 1
                _simTreeAge += 1
                continue

            
                                       
                                            
        
    else:
        print("No tree of this type!")
        print(_treeType)
        break

In [160]:
transformedTemplate

,plotID,farmerName,treeType,numCuerdas,status,startYear,deathYear
0,0,Estefanía Cazares,catuai,9,plant,2011,2028
1,0,Estefanía Cazares,catuai,9,replant,2029,2046
2,0,Estefanía Cazares,catuai,9,prune,2037,2050
3,1,Aldo Linares,catuai,1,plant,2018,2035
4,1,Aldo Linares,catuai,1,prune,2026,2039
...,...,...,...,...,...,...,...
362,102,Cynthia Magaña Palomo,catura,7,prune,2046,2058
363,103,Cynthia Magaña Palomo,e14,7,plant,2016,2031
364,103,Cynthia Magaña Palomo,e14,7,prune,2024,2035
365,103,Cynthia Magaña Palomo,e14,7,replant,2032,2051
